In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2024-02-21 02:21:46.458897: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 02:21:46.459030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 02:21:46.606757: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
display(df.head())
print(df.shape)

/tmp/ipykernel_33/3152433989.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])


,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


(54214, 4)


In [21]:
'''df_test=pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt", sep=':::',names=['ID','TITLE','GENRE','DESCRIPTIONS'])
display(train_data.head())
print(test_data.shape)'''

'df_test=pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt", sep=\':::\',names=[\'ID\',\'TITLE\',\'GENRE\',\'DESCRIPTIONS\'])\ndisplay(train_data.head())\nprint(test_data.shape)'

In [5]:
df[0:1]

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...


In [6]:
df

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...
...,...,...,...,...
54209,54210,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on ...
54210,54211,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The sist...
54211,54212,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about g..."
54212,54213,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and hav...


In [7]:
df.isna().sum()

ID             0
TITLE          0
GENRE          0
DESCRIPTION    0
dtype: int64

Data pre processing

#Step 1 converting into lower cases, so that two words with such as Ball and ball will be considered as singel token


In [10]:
import re
def clean_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [11]:
df['DESCRIPTION_1']=df['DESCRIPTION'].apply(clean_text)

In [16]:
df['DESCRIPTION_1']

0         Listening in to a conversation between his do...
1         A brother and sister with a past incestuous r...
2         As the bus empties the students for their fie...
3         To help their unemployed father make ends mee...
4         The films title refers not only to the unreco...
                               ...                        
54209     This shortlived NBC live sitcom centered on B...
54210     The NEXT Generation of EXPLOITATION The siste...
54211     Ze bestaan echt is a standup comedy about gro...
54212     Walter and Vivian live in the country and hav...
54213     On Labor Day Weekend 1935 the most intense hu...
Name: DESCRIPTION_1, Length: 54214, dtype: object

In [32]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    new_text = [word for word in text.split() if word not in stop_words]
    return ' '.join(new_text)

Removing html tags if there are any

In [17]:
df['DESCRIPTION_1']=df['DESCRIPTION_1'].apply(remove_stopwords)

In [18]:
df['DESCRIPTION_1']

0        Listening conversation doctor parents 10yearol...
1        A brother sister past incestuous relationship ...
2        As bus empties students field trip Museum Natu...
3        To help unemployed father make ends meet Edith...
4        The films title refers unrecovered bodies grou...
                               ...                        
54209    This shortlived NBC live sitcom centered Bonin...
54210    The NEXT Generation EXPLOITATION The sisters K...
54211    Ze bestaan echt standup comedy growing facing ...
54212    Walter Vivian live country difficult time keep...
54213    On Labor Day Weekend 1935 intense hurricane ev...
Name: DESCRIPTION_1, Length: 54214, dtype: object

In [19]:
import spacy 
nlp=spacy.load('en_core_web_sm')

In [23]:
import re 
def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
df['DESCRIPTION_1']=df['DESCRIPTION_1'].apply(tokenize)

In [24]:
df['DESCRIPTION_1'].head()

0    [Listening, conversation, doctor, parents, 10y...
1    [A, brother, sister, past, incestuous, relatio...
2    [As, bus, empties, students, field, trip, Muse...
3    [To, help, unemployed, father, make, ends, mee...
4    [The, films, title, refers, unrecovered, bodie...
Name: DESCRIPTION_1, dtype: object

In [39]:
from nltk.stem.porter import PorterStemmer

In [40]:
ps=PorterStemmer()
def steam_words(text):
    return " ".join([ps.stem(word) for word in text])

In [41]:
df['DESCRIPTION_1'].apply(steam_words)

0        listen convers doctor parent 10yearold oscar l...
1        A brother sister past incestu relationship cur...
2        As bu empti student field trip museum natur hi...
3        To help unemploy father make end meet edith tw...
4        the film titl refer unrecov bodi ground zero a...
                               ...                        
54209    thi shortliv nbc live sitcom center bonino wor...
54210    the next gener exploit the sister kapa bay sor...
54211    Ze bestaan echt standup comedi grow face fear ...
54212    walter vivian live countri difficult time keep...
54213    On labor day weekend 1935 intens hurrican ever...
Name: DESCRIPTION_1, Length: 54214, dtype: object

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Join the lists into strings
df['DESCRIPTION_1'] = df['DESCRIPTION_1'].apply(lambda x: ' '.join(x))

# Now you can fit and transform the data with TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['DESCRIPTION_1']).toarray()

In [43]:
print(tfidf)
print(tfidf.get_feature_names_out())

TfidfVectorizer()
['00' '000' '000km' ... 'żpleasure' 'żte' 'żwhere']


In [46]:
df.head()

,ID,TITLE,GENRE,DESCRIPTION,DESCRIPTION_1
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,Listening conversation doctor parents 10yearol...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...,A brother sister past incestuous relationship ...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...,As bus empties students field trip Museum Natu...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...,To help unemployed father make ends meet Edith...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...,The films title refers unrecovered bodies grou...


In [48]:
from sklearn.model_selection import train_test_split
X=df['DESCRIPTION_1']
y=df['GENRE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
X.head()

0    Listening conversation doctor parents 10yearol...
1    A brother sister past incestuous relationship ...
2    As bus empties students field trip Museum Natu...
3    To help unemployed father make ends meet Edith...
4    The films title refers unrecovered bodies grou...
Name: DESCRIPTION_1, dtype: object

In [50]:
y.head()

0        drama 
1     thriller 
2        adult 
3        drama 
4        drama 
Name: GENRE, dtype: object

In [51]:
y_train.head()

19253           drama 
12261           drama 
37143     documentary 
21965     documentary 
7033           comedy 
Name: GENRE, dtype: object

In [59]:
y_train.shape

(43371,)

In [52]:
X_train.head()

19253    Sex Betrayal Seduction Manipulation Addiction ...
12261    Autistic teen Tony friendship Cameron boy stil...
37143    The onehour special examines socially ethnical...
21965    This personal narrative documentary uses bicyc...
7033     The easygoing village policeman Artturi Sakari...
Name: DESCRIPTION_1, dtype: object

In [63]:
X_train.shape

(43371,)

In [53]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y_train_label=label_encoder.fit_transform(y_train)
y_test_label=label_encoder.fit_transform(y_test)

In [58]:
y_train_label.shape

(43371,)

In [60]:
vectorizer = TfidfVectorizer(max_features=100)  
X_train_vectorized = vectorizer.fit_transform(X_train).toarray()
print(X_train_vectorized)
X_test_vectorized = vectorizer.transform(X_test)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.57274913 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [62]:
X_train_vectorized.shape

(43371, 100)

In [65]:
max_tokens = df['DESCRIPTION_1'].apply(lambda x: len(word_tokenize(x))).max()
max_tokens

1480

In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split